# Restaurant Menu Complexity Project: Data Loading & Exploration

## Project Overview
**Research Question**: Does menu complexity (number of items) affect restaurant ratings?

**Hypothesis**: Simpler menus lead to higher ratings due to better execution and focus.

**Dataset**: Yelp Open Dataset
- ~150K businesses
- 6.9M reviews
- Multiple cities across US and Canada

## Objectives for This Notebook
1. Load Yelp JSON datasets
2. Filter to sit-down restaurants only
3. Apply quality thresholds
4. Explore data structure and availability
5. Save cleaned dataset for next steps

---

In [ ]:
# Cell 1: Imports and Setup
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print("Imports loaded successfully")

In [ ]:
# Cell 2: Helper Functions
def load_yelp_json(filepath, nrows=None):
    """
    Load Yelp JSON dataset (one JSON object per line)
    
    Parameters:
    -----------
    filepath : str
        Path to the JSON file
    nrows : int, optional
        Number of rows to load (None loads all)
    
    Returns:
    --------
    pd.DataFrame
    """
    data = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if nrows and i >= nrows:
                break
            data.append(json.loads(line))
    return pd.DataFrame(data)

def is_restaurant(categories):
    """Check if business is a restaurant"""
    if pd.isna(categories):
        return False
    categories_lower = categories.lower()
    restaurant_keywords = ['restaurant', 'food', 'cafe', 'bistro', 'eatery', 'diner']
    return any(keyword in categories_lower for keyword in restaurant_keywords)

def should_exclude(categories):
    """Exclude fast food, bars, convenience stores"""
    if pd.isna(categories):
        return True
    categories_lower = categories.lower()
    exclude_keywords = [
        'fast food', 'food truck', 'gas station', 
        'convenience store', 'bar', 'nightlife'
    ]
    return any(keyword in categories_lower for keyword in exclude_keywords)

print("Helper functions defined")

In [ ]:
# Cell 3: Load Business Data
print("Loading business data...")
print("(This may take 1-2 minutes)")

business_df = load_yelp_json('data/raw/yelp_academic_dataset_business.json')

print(f"\nLoaded {len(business_df):,} businesses")
print(f"Shape: {business_df.shape}")
print(f"\nColumns: {business_df.columns.tolist()}")

In [ ]:
# Cell 4: Explore Business Data Structure
print("=== BUSINESS DATA SAMPLE ===\n")
print(business_df.head(3))

print("\n=== DATA TYPES ===\n")
print(business_df.dtypes)

print("\n=== MISSING VALUES ===\n")
missing = business_df.isnull().sum()
missing_pct = (missing / len(business_df)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Percentage': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0].sort_values('Percentage', ascending=False))

print("\n=== BASIC STATISTICS ===\n")
print(business_df[['stars', 'review_count']].describe())

In [ ]:
# Cell 5: Explore Categories
print("=== SAMPLE CATEGORIES ===\n")
print(business_df['categories'].head(20))

print("\n=== CATEGORY STATISTICS ===")
print(f"Businesses with categories: {business_df['categories'].notna().sum():,}")
print(f"Businesses without categories: {business_df['categories'].isna().sum():,}")

In [ ]:
# Cell 6: Filter to Restaurants
print("Filtering to restaurants...")

# Apply restaurant filter
business_df['is_restaurant'] = business_df['categories'].apply(is_restaurant)
restaurants_df = business_df[business_df['is_restaurant']].copy()

print(f"Found {len(restaurants_df):,} restaurant businesses")

# Exclude unwanted types
restaurants_df['exclude'] = restaurants_df['categories'].apply(should_exclude)
restaurants_df = restaurants_df[~restaurants_df['exclude']].copy()

print(f"After exclusions: {len(restaurants_df):,} restaurants")

In [ ]:
# Cell 7: Apply Quality Filters
print("Applying quality filters...")

initial_count = len(restaurants_df)

# Filter criteria
restaurants_df = restaurants_df[
    (restaurants_df['review_count'] >= 20) &  # Established businesses
    (restaurants_df['is_open'] == 1) &        # Currently open
    (restaurants_df['stars'] > 0)             # Has ratings
].copy()

final_count = len(restaurants_df)
removed = initial_count - final_count

print(f"Restaurants before filters: {initial_count:,}")
print(f"Restaurants after filters: {final_count:,}")
print(f"Removed: {removed:,} ({removed/initial_count*100:.1f}%)")

In [ ]:
# Cell 8: Explore Geographic Distribution
print("=== GEOGRAPHIC DISTRIBUTION ===\n")

print("Top 15 Cities:")
print(restaurants_df['city'].value_counts().head(15))

print("\n\nStates/Provinces:")
print(restaurants_df['state'].value_counts())

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top cities
top_cities = restaurants_df['city'].value_counts().head(15)
axes[0].barh(range(len(top_cities)), top_cities.values)
axes[0].set_yticks(range(len(top_cities)))
axes[0].set_yticklabels(top_cities.index)
axes[0].set_xlabel('Number of Restaurants')
axes[0].set_title('Top 15 Cities by Restaurant Count')
axes[0].invert_yaxis()

# States
state_counts = restaurants_df['state'].value_counts()
axes[1].bar(range(len(state_counts)), state_counts.values)
axes[1].set_xticks(range(len(state_counts)))
axes[1].set_xticklabels(state_counts.index, rotation=45)
axes[1].set_ylabel('Number of Restaurants')
axes[1].set_title('Restaurants by State/Province')

plt.tight_layout()
plt.savefig('outputs/figures/geographic_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 9: Explore Rating Distribution
print("=== RATING DISTRIBUTION ===\n")
print(restaurants_df['stars'].describe())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(restaurants_df['stars'], bins=20, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Rating (Stars)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Restaurant Ratings')
axes[0].axvline(restaurants_df['stars'].mean(), color='red', 
                linestyle='--', label=f'Mean: {restaurants_df["stars"].mean():.2f}')
axes[0].legend()

# Box plot
axes[1].boxplot(restaurants_df['stars'])
axes[1].set_ylabel('Rating (Stars)')
axes[1].set_title('Restaurant Rating Box Plot')

plt.tight_layout()
plt.savefig('outputs/figures/rating_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Cell 10: Explore Review Count Distribution
print("=== REVIEW COUNT DISTRIBUTION ===\n")
print(restaurants_df['review_count'].describe())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram (log scale)
axes[0].hist(np.log10(restaurants_df['review_count']), bins=30, 
             edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Log10(Review Count)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Review Counts (Log Scale)')

# Scatter: Review Count vs Rating
axes[1].scatter(restaurants_df['review_count'], restaurants_df['stars'], 
                alpha=0.3, s=10)
axes[1].set_xlabel('Review Count')
axes[1].set_ylabel('Rating (Stars)')
axes[1].set_title('Review Count vs Rating')
axes[1].set_xscale('log')

plt.tight_layout()
plt.savefig('outputs/figures/review_count_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Correlation
corr = restaurants_df[['stars', 'review_count']].corr()
print("\nCorrelation between stars and review_count:")
print(corr)

In [ ]:
# Cell 11: Save Processed Data
print("Saving processed data...")

# Save full dataset
restaurants_df.to_csv('data/processed/restaurants_filtered.csv', index=False)
print(f"Saved {len(restaurants_df):,} restaurants to data/processed/restaurants_filtered.csv")

# Save summary statistics
summary_stats = {
    'total_restaurants': len(restaurants_df),
    'total_reviews': restaurants_df['review_count'].sum(),
    'avg_rating': restaurants_df['stars'].mean(),
    'median_rating': restaurants_df['stars'].median(),
    'avg_reviews_per_restaurant': restaurants_df['review_count'].mean(),
    'cities': restaurants_df['city'].nunique(),
    'states': restaurants_df['state'].nunique(),
    'date_processed': datetime.now().strftime('%Y-%m-%d')
}

summary_df = pd.DataFrame([summary_stats])
summary_df.to_csv('data/processed/summary_stats.csv', index=False)
print("Saved summary statistics")

# Display summary
print("\n=== PROCESSING COMPLETE ===")
print(f"Total restaurants saved: {len(restaurants_df):,}")
print(f"Cities represented: {restaurants_df['city'].nunique()}")
print(f"Average rating: {restaurants_df['stars'].mean():.2f}")
print(f"Average reviews per restaurant: {restaurants_df['review_count'].mean():.1f}")

In [ ]:
print("=== REVIEW COUNT DISTRIBUTION ===\n")
print(restaurants_df['review_count'].describe())

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram (log scale)
axes[0].hist(np.log10(restaurants_df['review_count']), bins=30, 
             edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Log10(Review Count)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Review Counts (Log Scale)')

# Scatter: Review Count vs Rating
axes[1].scatter(restaurants_df['review_count'], restaurants_df['stars'], 
                alpha=0.3, s=10)
axes[1].set_xlabel('Review Count')
axes[1].set_ylabel('Rating (Stars)')
axes[1].set_title('Review Count vs Rating')
axes[1].set_xscale('log')

plt.tight_layout()
plt.savefig('outputs/review_count_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Correlation
corr = restaurants_df[['stars', 'review_count']].corr()
print("\nCorrelation between stars and review_count:")
print(corr)

In [ ]:
print("Saving processed data...")

# Save full dataset
restaurants_df.to_csv('data/restaurants_filtered.csv', index=False)
print(f"Saved {len(restaurants_df):,} restaurants to data/restaurants_filtered.csv")

# Save summary statistics
summary_stats = {
    'total_restaurants': len(restaurants_df),
    'total_reviews': restaurants_df['review_count'].sum(),
    'avg_rating': restaurants_df['stars'].mean(),
    'median_rating': restaurants_df['stars'].median(),
    'avg_reviews_per_restaurant': restaurants_df['review_count'].mean(),
    'cities': restaurants_df['city'].nunique(),
    'states': restaurants_df['state'].nunique(),
    'date_processed': datetime.now().strftime('%Y-%m-%d')
}

summary_df = pd.DataFrame([summary_stats])
summary_df.to_csv('data/summary_stats.csv', index=False)
print("Saved summary statistics")

---

## Summary

### Dataset Characteristics
- **Total Restaurants**: 17,625
- **Cities Represented**: 675
- **Average Rating**: 3.74
- **Date Range**: 2005-02-16 to 2022-01-19

### Key Observations
1. Philidelphia has a significantly more number of resturerant reviews vs every other city. 
2. Nothing significantly wrong with data set provided 
3. Within the top 15 cities ranked, Philidelphia holds over 34% of the ratings given. Might need to look    more into what goes on in Phili. 

### Next Steps
In the next notebook, we'll:
1. Engineer menu complexity proxy features
2. Extract signals from business attributes
3. Mine review text for menu mentions
4. Create treatment variable (simple vs complex menu)

---